In [2]:
from __future__ import division
from __future__ import print_function

import os
import sys
import json
import pprint
import warnings
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import torch.nn.utils.prune as prune
from torch.optim import lr_scheduler
import torchvision.transforms as transforms

import sklearn.metrics
from collections import OrderedDict

from torch.autograd import Variable
from PIL import Image

# Data Retriver

In [3]:
class Parser():
  def __init__(self, model = "densenet", optimizer = "adam", lr = 0.0001, weight_decay = 0.0, drop_rate = 0.0, epochs = 165,\
               batch_size = 16, workers = 8, seed = 123456, tag = "", toy = False, save_path ="./", scale = 224, horizontal_flip = True,\
               verbose = True, scratch = True, train_weighted = True, valid_weighted = True, size = None):
    self.model = model; self.optimizer = optimizer; self.lr = lr; self.weight_decay = weight_decay; self.drop_rate = drop_rate
    self.epochs = epochs; self.batch_size = batch_size; self.workers = workers; self.seed = seed; self.tag = tag; self.toy = toy
    self.save_path = save_path; self.scale = scale; self.horizontal_flip = horizontal_flip; self.verbose = verbose; self.scratch = scratch
    self.train_weighted = train_weighted; self.valid_weighted = valid_weighted; self.size = size

class Dataset(data.Dataset):
    def __init__(self, args, data_split):
        super(Dataset, self).__init__()
        tag = ""
        if args.tag:
          tag = "_"+args.tag
        df = pd.read_csv(os.path.join("../input/latestdata/%s%s.csv" % (data_split, tag)))
        df['Path'] = df['Image Index']
        
        # Có thử nghiệm hay không? frac = tỉ lệ data lấy ra từ dataset
        if args.toy:
            df = df.sample(frac=0.01)

        self.df = df
        self.img_paths = df["Path"].tolist()
        self.pathologies = [col for col in df.columns.values if (col != "Path" and col != "No Finding" and col != 'Image Index')]
        self.labels = df[self.pathologies].to_numpy().astype(int)
        
        self.n_classes = self.labels.shape[1]

        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        if data_split == "train":
            transforms_lst = [
                transforms.Resize((args.scale, args.scale)),
                transforms.RandomHorizontalFlip() if args.horizontal_flip else None,
                transforms.ToTensor(),
                normalize,
            ]
            self.transform = transforms.Compose([t for t in transforms_lst if t])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((args.scale, args.scale)),
                transforms.ToTensor(),
                normalize,
            ])
        self.df = df
                
        if (data_split == "train" and args.train_weighted) or (data_split == "valid" and args.valid_weighted):
            self.get_weights(args, data_split)


    def get_weights(self, args, data_split):

        self.use_gpu = torch.cuda.is_available()
        p_count = (self.labels == 1).sum(axis = 0)
        self.p_count = p_count
        n_count = (self.labels == 0).sum(axis = 0)
        total = p_count + n_count

        # invert *opposite* weights to obtain weighted loss
        # (positives weighted higher, all weights same across batches, and p_weight + n_weight == 1)
        p_weight = n_count / total
        n_weight = p_count / total

        self.p_weight_loss = Variable(torch.FloatTensor(p_weight), requires_grad=False)
        self.n_weight_loss = Variable(torch.FloatTensor(n_weight), requires_grad=False)

        print ("Positive %s Loss weight:" % data_split, self.p_weight_loss.data.numpy())
        print ("Negative %s Loss weight:" % data_split, self.n_weight_loss.data.numpy())
        random_loss = sum((p_weight[i] * p_count[i] + n_weight[i] * n_count[i]) *\
                                               -np.log(0.5) / total[i] for i in range(self.n_classes)) / self.n_classes
        print ("Random %s Loss:" % data_split, random_loss)


    def __getitem__(self, index):
        img = Image.open(self.img_paths[index]).convert("RGB")
        label = self.labels[index]

        return self.transform(img), torch.LongTensor(label)

    def __len__(self):
        return len(self.img_paths)

    def weighted_loss(self, preds, target, epoch=1):

        weights = target.type(torch.FloatTensor) * (self.p_weight_loss.expand_as(target)) + \
                  (target == 0).type(torch.FloatTensor) * (self.n_weight_loss.expand_as(target))
        if self.use_gpu:
            weights = weights.cuda()
        loss = 0.0
        for i in range(self.n_classes):
            loss += nn.functional.binary_cross_entropy_with_logits(preds[:,i], target[:,i], weight=weights[:,i])
        return loss / self.n_classes


def evaluate(gts, probabilities, pathologies, use_only_index = None):
    assert(np.all(probabilities >= 0) == True)
    assert(np.all(probabilities <= 1) == True)

    def compute_metrics_for_class(i):
         p, r, t = sklearn.metrics.precision_recall_curve(gts[:, i], probabilities[:, i])
         PR_AUC = sklearn.metrics.auc(r, p)
         ROC_AUC = sklearn.metrics.roc_auc_score(gts[:, i], probabilities[:, i])
         F1 = sklearn.metrics.f1_score(gts[:, i], preds[:, i])
         acc = sklearn.metrics.accuracy_score(gts[:, i], preds[:, i])
         count = np.sum(gts[:, i])
         return PR_AUC, ROC_AUC, F1, acc, count

    PR_AUCs = []
    ROC_AUCs = []
    F1s = []
    accs = []
    counts = []
    preds = probabilities >= 0.5

    classes = [use_only_index] if use_only_index is not None else range(len(gts[0]))

    for i in classes:
        try:
            PR_AUC, ROC_AUC, F1, acc, count = compute_metrics_for_class(i)
        except ValueError:
            continue
        PR_AUCs.append(PR_AUC)
        ROC_AUCs.append(ROC_AUC)
        F1s.append(F1)
        accs.append(acc)
        counts.append(count)
        print('Class: {!s} Count: {:d} PR AUC: {:.4f} ROC AUC: {:.4f} F1: {:.3f} Acc: {:.3f}'.format(pathologies[i], count, PR_AUC, ROC_AUC, F1, acc))

    avg_PR_AUC = np.average(PR_AUCs)
    avg_ROC_AUC = np.average(ROC_AUCs, weights=counts)
    avg_F1 = np.average(F1s, weights=counts)

    print('Avg PR AUC: {:.3f}'.format(avg_PR_AUC))
    print('Avg ROC AUC: {:.3f}'.format(avg_ROC_AUC))
    print('Avg F1: {:.3f}'.format(avg_F1))
    return avg_PR_AUC, avg_ROC_AUC, avg_F1


def loader_to_gts(data_loader):
    gts = []
    for (inputs, labels) in data_loader:
        for label in labels.cpu().numpy().tolist():
            gts.append(label)
    gts = np.array(gts)
    return gts


def load_data(args):

    train_dataset = Dataset(args, "train")
    valid_dataset = Dataset(args, "valid")

    train_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=args.batch_size, shuffle=True,
            num_workers=args.workers, pin_memory=True, sampler=None)
    print("train shape la: ",train_dataset.__len__())
    print("valid shape la: ", valid_dataset.__len__())

    valid_loader = torch.utils.data.DataLoader(
            valid_dataset, batch_size=args.batch_size, shuffle=False,
            num_workers=args.workers, pin_memory=True, sampler=None)

    return train_loader, valid_loader


# Densenet configuration

In [4]:
# Define các hằng số

n_patho = 14
model_path = {
    'densenet121': '../input/densenet121/densenet121.pth'
}

# Tùy chỉnh Densenet
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm-1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu-1', nn.ReLU(inplace=True)),
        self.add_module('conv-1', nn.Conv2d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm-2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu-2', nn.ReLU(inplace=True)),
        self.add_module('conv-2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """
    def __init__(self, num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16),
                 bn_size=4, drop_rate=0, num_classes=1000):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        self.gap = nn.AdaptiveAvgPool2d(1)

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

    def forward(self, x):
#        features = self.features(x)
        f = self.features.conv0(x)
        f = self.features.norm0(f)
        f = self.features.relu0(f)
        f = self.features.pool0(f)
        
        # 3 blocks and 2 transitions
        f = self.features.denseblock1(f)
        f = self.features.transition1(f)
        f = self.features.denseblock2(f)
        f = self.features.transition2(f)
        f = self.features.denseblock3(f)
        
        out = F.relu(f, inplace=True)
        out = self.gap(out).view(f.size(0), -1)
        out = self.classifier(out)
        return out
    
# def densenet121(pretrained=True, **kwargs):
#     r"""Densenet-121 model from
#     `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#     """
#     model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16), bn_size=4, drop_rate = 0, num_classes=1000)
#     if pretrained:
#         model.load_state_dict(torch.load(model_path['densenet121']), strict=False)
#     return model

def densenet121(pretrained = True):
    if(pretrained == True):
        model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
    else:
        model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)
    def __init__(self):
        super().__init__()
        self.net = model

        
        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        self.gap = nn.AdaptiveAvgPool2d(1)

        # Linear layer
        self.pooling = nn.AvgPool2d()
        self.classifier = nn.Linear(32,14)
    def forward(self, x):
        f = self.net.features.conv0(x)
        f = self.net.features.norm0(f)
        f = self.net.features.relu0(f)
        f = self.net.features.pool0(f)
        
        # 3 blocks and 2 transitions
        f = self.net.features.denseblock1(f)
        f = self.net.features.transition1(f)
        f = self.net.features.denseblock2(f)
        f = self.net.features.transition2(f)
        f = self.net.features.denseblock3(f)
        
        # classification block containing a global average pooling layer followed by a fully connected layer 
        self.pooling(f)
        self.classifier(f)
        return out

In [5]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
model.features

In [6]:
"""
Train script for CheXNet
"""

def transform_data(data, use_gpu, train=False):
    inputs, labels = data
    labels = labels.type(torch.FloatTensor)
    if use_gpu is True:
        inputs = inputs.cuda()
        labels = labels.cuda()
    inputs = Variable(inputs, requires_grad=False, volatile=not train)
    labels = Variable(labels, requires_grad=False, volatile=not train)
    return inputs, labels


def train_epoch(epoch, args, model, loader, criterion, optimizer):
    model.train()
    batch_losses = []
    for batch_idx, data in enumerate(loader):
        inputs, labels = transform_data(data, True, train=True)
        optimizer.zero_grad()
        outputs = model(inputs)
        print('shape labels: ', labels.shape)
        print('shape output: ', outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print("Epoch: {:d} Batch: {:d} ({:d}) Train Loss: {:.6f}".format(
            epoch, batch_idx, args.batch_size, loss.data))
        sys.stdout.flush()
        batch_losses.append(loss.data)
    #train_loss = np.mean(batch_losses)
    train_loss = torch.stack(batch_losses).mean().item()
    print("Training Loss: {:.6f}".format(train_loss))
    return train_loss


def test_epoch(model, loader, criterion, epoch=1):
    """
    Returns: (AUC, ROC AUC, F1, validation loss)
    """
    model.eval()
    test_losses = []
    outs = []
    gts = []
    for data in loader:
        for gt in data[1].numpy().tolist():
            gts.append(gt)
        inputs, labels = transform_data(data, True, train=False)
        outputs = model(inputs)
        loss = criterion(outputs, labels, epoch=epoch)
        test_losses.append(loss.data)
        out = torch.sigmoid(outputs).data.cpu().numpy()
        outs.extend(out)
    #avg_loss = np.mean(test_losses)
    avg_loss = torch.stack(test_losses).mean().item()
    print("Validation Loss: {:.6f}".format(avg_loss))
    outs = np.array(outs)
    gts = np.array(gts)
    return evaluate(gts, outs, loader.dataset.pathologies) + (avg_loss,)


def get_loss(dataset, weighted):

    criterion = nn.MultiLabelSoftMarginLoss()

    def loss(preds, target, epoch):

        if weighted:

            return dataset.weighted_loss(preds, target, epoch=epoch)

        else:

            return criterion(preds, target)

    return loss


def run(args):

    use_gpu = torch.cuda.is_available()
    model = densenet121(pretrained=True, num_classes = 14)

    train, val = load_data(args)
    print("train.shape = {}",len(train))
    print("val.shape = {}", len(val))
    nclasses = train.dataset.n_classes
    print("Number of classes:", nclasses)
    
    if args.model == "densenet":
        model = densenet121(pretrained=True, num_classes = nclasses)
    else:
        print("{} is not a valid model.".format(args.model))
    
    if use_gpu:
        model = model.cuda()

    #train_criterion = get_loss(train.dataset, args.train_weighted)
    train_criterion = F.binary_cross_entropy
    val_criterion = get_loss(val.dataset, args.valid_weighted)

    if args.optimizer == "adam":
        optimizer = optim.Adam(
                       filter(lambda p: p.requires_grad, model.parameters()),
                       lr=args.lr,
                       weight_decay=args.weight_decay)
    elif args.optimizer == "rmsprop":
        optimizer = optim.RMSprop(
                       filter(lambda p: p.requires_grad, model.parameters()),
                       lr=args.lr,
                       weight_decay=args.weight_decay)
    else:
        print("{} is not a valid optimizer.".format(args.optimizer))

    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, threshold=0.001, factor=0.1)
    best_model_wts, best_loss = model.state_dict(), float("inf")

    counter = 0
    for epoch in range(1, args.epochs + 1):
        print("Epoch {}/{}".format(epoch, args.epochs))
        print("-" * 10)
        train_loss = train_epoch(epoch, args, model, train,train_criterion, optimizer)
        _, epoch_auc, _, valid_loss = test_epoch(model, val, val_criterion, epoch)
        scheduler.step(valid_loss)

        if (valid_loss < best_loss):
            best_loss = valid_loss
            best_model_wts = model.state_dict()
            counter = 0
        else:
            counter += 1

        if counter > 3:
            print("ran out of patience....")
            break

        torch.save(best_model_wts, os.path.join(args.save_path, "val%f_train%f_epoch%d" % (valid_loss, train_loss, epoch)))

    print("Best Validation Loss:", best_loss)

if __name__ == "__main__":
    """
    Usage
        Download the images data at https://nihcc.app.box.com/v/ChestXray-NIHCC
        To train on the original labels:
            python train.py --save_path run_dir --model densenet --batch_size 8 --horizontal_flip --epochs 10 --lr 0.0001 --train_weighted --valid_weighted --scale 512
        To train on the relabels:
            python train.py --save_path run_dir --model densenet --batch_size 8 --horizontal_flip --epochs 10 --lr 0.0001 --train_weighted --valid_weighted --scale 512 --tag relabeled
    """
    p = Parser()
    run(p)

